<a href="https://colab.research.google.com/github/Rahi27/Rahi_Projects/blob/main/Images_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === Siamese embedding + inference (MobileNetV2 + contrastive loss) ===
import os
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [3]:
# ---------- 1) config ----------
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 3
MARGIN = 1.0  # contrastive margin
ZIP_PATH = "/content/archive.zip"          # your zip (if exists)
EXTRACT_PATH = "/content/archive"          # target extract folder
CSV_PATH = "/content/sketch_photo_dataset(1).csv"
PHOTOS_FOLDER = os.path.join(EXTRACT_PATH, "photos")
SKETCHES_FOLDER = os.path.join(EXTRACT_PATH, "sketches")

In [4]:
# ---------- 2) extract zip if needed ----------
import zipfile
import os

zip_path = "/content/archive.zip"
extract_path = "/content/archive"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Files extracted to:", extract_path)

✅ Files extracted to: /content/archive


In [5]:
# ---------- 3) load CSV and prepare splits ----------
df = pd.read_csv(CSV_PATH)
print("CSV sample:\n", df.head())


# map labels if needed (adjust mapping to your CSV)
label_map = {'yes': 1, 'no': 0}   # change if your labels differ
if df['label'].dtype == object:
    df['label'] = df['label'].replace(label_map)
df['label'] = df['label'].astype(int)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
print("Train/test sizes:", len(train_df), len(test_df))

CSV sample:
      photo_image       sketch_image label
0  m1-024-01.jpg  m1-037-01-sz1.jpg    no
1  m1-024-01.jpg   m-079-01-sz1.jpg    no
2  m1-024-01.jpg   m-089-01-sz1.jpg    no
3  m1-024-01.jpg   m-075-01-sz1.jpg    no
4  m1-024-01.jpg  F2-018-01-sz1.jpg    no
Train/test sizes: 28275 7069


/tmp/ipython-input-690002582.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace(label_map)


In [6]:
# ---------- 4) image preprocess (with light augmentation option) ----------
def load_and_preprocess(path, augment=False):
    try:
        img = load_img(path, target_size=IMG_SIZE)
        img = img_to_array(img) / 255.0
    except Exception as e:
        # fallback blank image
        print("Error loading:", path, "->", e)
        img = np.zeros((*IMG_SIZE, 3), dtype=np.float32)

    if augment:
        # simple augmentation: random flip + brightness jitter
        if np.random.rand() < 0.5:
            img = np.fliplr(img)
        b = 0.9 + 0.2 * np.random.rand()
        img = np.clip(img * b, 0.0, 1.0)
    return img.astype(np.float32)

In [7]:
# ---------- 5) generator yielding (sketch_batch, photo_batch), labels ----------
def pair_generator(df, batch_size=BATCH_SIZE, augment=False):
    n = len(df)
    while True:
        df_shuffled = df.sample(frac=1).reset_index(drop=True)
        for start in range(0, n, batch_size):
            end = min(start + batch_size, n)
            batch = df_shuffled.iloc[start:end]
            A, B, Y = [], [], []
            for _, row in batch.iterrows():
                sketch_path = os.path.join(SKETCHES_FOLDER, row['sketch_image'])
                photo_path = os.path.join(PHOTOS_FOLDER, row['photo_image'])
                A.append(load_and_preprocess(sketch_path, augment=augment))
                B.append(load_and_preprocess(photo_path, augment=augment))
                Y.append(row['label'])
            yield (np.array(A), np.array(B)), np.array(Y, dtype=np.float32)


In [8]:
# ---------- 6) build embedding network (shared) ----------
def build_embedding_model(input_shape=(*IMG_SIZE, 3), embedding_dim=128):
    base = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=input_shape)
    for layer in base.layers:
        layer.trainable = False
    x = base.output
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(embedding_dim)(x)
    # L2 normalize embeddings so cosine similarity is just dot product
    x = Lambda(lambda t: K.l2_normalize(t, axis=1), name='l2_norm')(x)
    return Model(inputs=base.input, outputs=x, name='embedding_net')

embedding_net = build_embedding_model()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:

# ---------- 7) siamese model that outputs euclidean distance ----------
inp_a = Input(shape=(*IMG_SIZE, 3))
inp_b = Input(shape=(*IMG_SIZE, 3))
emb_a = embedding_net(inp_a)
emb_b = embedding_net(inp_b)

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

distance = Lambda(euclidean_distance, name='euclid_dist')([emb_a, emb_b])
siamese_model = Model(inputs=[inp_a, inp_b], outputs=distance)

In [10]:
# ---------- 8) contrastive loss ----------
def contrastive_loss(margin=MARGIN):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, y_pred.dtype)
        # y_true==1 -> positive pair -> minimize d^2
        # y_true==0 -> negative pair -> minimize max(margin - d, 0)^2
        pos = y_true * tf.square(y_pred)
        neg = (1 - y_true) * tf.square(tf.maximum(margin - y_pred, 0.0))
        return tf.reduce_mean(pos + neg)
    return loss

siamese_model.compile(optimizer=Adam(1e-4), loss=contrastive_loss(MARGIN))
siamese_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_net       │ (None, 128)       │  2,618,816 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ euclid_dist         │ (None, 1)         │          0 │ embedding_net[0]… │
│ (Lambda)            │                   │            │ embedding_net[1]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,618,816 (9.99 MB)

 Trainable params: 360,832 (1.38 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
# ---------- 9) train ----------
train_gen = pair_generator(train_df, batch_size=BATCH_SIZE, augment=True)
val_gen = pair_generator(test_df, batch_size=BATCH_SIZE, augment=False)
steps_per_epoch = max(1, len(train_df) // BATCH_SIZE)
val_steps = max(1, len(test_df) // BATCH_SIZE)

history = siamese_model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=val_steps,
    epochs=EPOCHS
)


Epoch 1/3
883/883 ━━━━━━━━━━━━━━━━━━━━ 105s 92ms/step - loss: 0.0035 - val_loss: 0.0153
Epoch 2/3
883/883 ━━━━━━━━━━━━━━━━━━━━ 88s 84ms/step - loss: 0.0033 - val_loss: 0.0179
Epoch 3/3
883/883 ━━━━━━━━━━━━━━━━━━━━ 75s 85ms/step - loss: 0.0034 - val_loss: 0.0089


In [12]:
# ---------- 10) build gallery embeddings (photos) ----------
photo_files = [f for f in os.listdir(PHOTOS_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
print("Number of photos in gallery:", len(photo_files))

# batch predict gallery embeddings for speed
photo_files = [f for f in os.listdir(PHOTOS_FOLDER) if f.lower().endswith(('.jpg','.jpeg','.png'))]
print("Number of photos in gallery:", len(photo_files))

# batch predict gallery embeddings for speed
def compute_embeddings(file_list, batch=64):
    embeddings = []
    names = []
    for i in range(0, len(file_list), batch):
        batch_files = file_list[i:i+batch]
        imgs = []
        for fname in batch_files:
            p = os.path.join(PHOTOS_FOLDER, fname)
            imgs.append(load_and_preprocess(p, augment=False))
        imgs = np.array(imgs)
        embs = embedding_net.predict(imgs, verbose=0)   # normalized embeddings
        embeddings.append(embs)
        names.extend(batch_files)
    return np.vstack(embeddings), names

gallery_embeddings, gallery_names = compute_embeddings(photo_files)
# gallery_embeddings are L2-normalized per-row

Number of photos in gallery: 188
Number of photos in gallery: 188


In [13]:
# ---------- 11) matching function using cosine similarity (dot product) ----------
def find_best_matches_for_sketch(sketch_path, top_k=5):
    s_img = load_and_preprocess(sketch_path, augment=False)
    s_emb = embedding_net.predict(np.expand_dims(s_img, axis=0), verbose=0)[0]  # (embedding_dim,)
    # cosine similarity since embeddings are L2-normalized
    sims = np.dot(gallery_embeddings, s_emb)   # shape (n_photos,)
    order = np.argsort(-sims)  # descending
    results = [(gallery_names[idx], float(sims[idx])) for idx in order[:top_k]]
    return results


In [14]:

# ---------- 12) test on a sketch ----------
test_sketch = os.path.join(SKETCHES_FOLDER, "m-063-01-sz1.jpg")   # change to your sketch file
top_matches = find_best_matches_for_sketch(test_sketch, top_k=5)
print("Top matches (filename, cosine-score):")
for fn, score in top_matches:
    print(fn, score)

Top matches (filename, cosine-score):
m-091-01.jpg 0.7470476627349854
m-078-01.jpg 0.746284008026123
m-088-01.jpg 0.7391812801361084
m1-027-01.jpg 0.728235125541687
m-064-01.jpg 0.7161684036254883
